In [19]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
import geopandas as gpd
import json
from rasterio.mask import mask
from shapely.geometry import box

In [20]:
#Pre widfire images     12 August 2020
NIR_pre = rasterio.open('T10SFF_20200812T184921_B08_10m.jp2')
SWIR_pre = rasterio.open('T10SFF_20200812T184921_B12_20m.jp2')

#Post wildfire images   11 October 2020
NIR_post = rasterio.open('T10SFF_20201011T185321_B0A_10m.jp2')
SWIR_post = rasterio.open('T10SFF_20201011T185321_B12_20m.jp2')

In [21]:
#Resample to apply arithmetic band operations
def resample(NIR_band, SWIR_band, output_name ):
    resample_factor = int(NIR_band.width / SWIR_band.width)
    resampled_SWIR  = SWIR_band.read(out_shape=(SWIR_band.count,SWIR_band.height * resample_factor,
                                          int(SWIR_band.width * resample_factor)),
                                          resampling=Resampling.bilinear)
    resampled_meta = SWIR_band.meta.copy()
    resampled_meta.update({"driver": "GTiff",
                        'height': resampled_SWIR.shape[1],
                        'width': resampled_SWIR.shape[2],
                        'transform': NIR_band.transform,
                        'crs': SWIR_band.crs})
    with rasterio.open(output_name, 'w', **resampled_meta) as clip:
            clip.write(resampled_SWIR)
            clip.close()

In [22]:
#Call resample function with a proper parameters
resample(NIR_pre, SWIR_pre, 'SWIR_pre_res.tif')
resample(NIR_post, SWIR_post, 'SWIR_post_res.tif')

In [23]:
#Clip function to select region of interest.
def clip(inputFile, outputFile):
    #Creating bounding box to clip ROI
    boundFrame = gpd.GeoDataFrame({'geometry': box(620000,3990000, 650000 , 4010000)}, index=[0], crs=inputFile.crs)
    jsonBox = [json.loads(boundFrame.to_json())['features'][0]['geometry']]
    clipped, clipped_transform = mask(dataset=inputFile, shapes=jsonBox, crop=True)
    clipped_meta = inputFile.meta.copy()
    clipped_meta.update({"driver": "GTiff",
                        'height': clipped.shape[1],
                        'width': clipped.shape[2],
                        'transform': clipped_transform,
                        'crs': inputFile.crs})
    
    #save as tif
    with rasterio.open(outputFile, 'w', **clipped_meta) as clip:
        clip.write(clipped)
        clip.close()

In [24]:
#Call clip functions with a proper parameters.

clip(NIR_pre, 'NIR_pre_clipped.tif')
clip(rasterio.open('SWIR_pre_res.tif'), 'SWIR_pre_clipped.tif')

clip(NIR_post, 'NIR_post_clipped.tif')
clip(rasterio.open('SWIR_post_res.tif'), 'SWIR_post_clipped.tif')

In [25]:
#Open pre wildfire clipped images
NIR_clipped_pre = rasterio.open('NIR_pre_clipped.tif')
SWIR_clipped_pre = rasterio.open('SWIR_pre_clipped.tif')

#Open ppost wildfire clipped images
NIR_clipped_post = rasterio.open('NIR_post_clipped.tif')
SWIR_clipped_post = rasterio.open('SWIR_post_clipped.tif')

In [12]:
#Numpy environment settings to apply arithmetic operations properly.
np.seterr(divide='ignore', invalid='ignore')

def NBR_Calc(NIR, SWIR, output):
    #Calculating Normalized Burn Ratio Index
    NBR = (NIR.read().astype(float) - SWIR.read().astype(float)) / (NIR.read() - SWIR.read())
    meta = NIR.meta.copy()
    meta.update(driver='GTiff')
    meta.update(dtype=rasterio.float32)

    with rasterio.open(output, 'w', **meta) as dst:
        dst.write(NBR.astype(rasterio.float32))
        dst.close()

In [13]:
#Calling NBR_Calc function to calculate pre and post wildfire.
NBR_Calc(NIR_clipped_pre, SWIR_clipped_pre, 'NBR_pre.tif')
NBR_Calc(NIR_clipped_post, SWIR_clipped_post, 'NBR_post.tif')

In [15]:
#Creating difference NBR values to detect how to change

def dnbr(preNBR, postNBR):
    dnbr = preNBR.read() - postNBR.read()
    meta = preNBR.meta.copy()
    meta.update(driver='GTiff')
    meta.update(dtype=rasterio.float32)

    with rasterio.open('dnbr.tif', 'w', **meta) as dst:
        dst.write(dnbr.astype(rasterio.float32))
        dst.close()

In [16]:
#Open NBR images
NBR_pre = rasterio.open('NBR_pre.tif')
NBR_post = rasterio.open('NBR_post.tif')

In [17]:
#Call dnbr function to calculate dnbr
dnbr(NBR_pre, NBR_post)

In [18]:
#open dnbr
dnbr = rasterio.open('dnbr.tif')